In [1]:
%run -i ./import_pageviews_sorted_by_time_spark.py './sorted_time/2007-12/pagecounts-20071210-*.gz'

[2018-05-18 03:48:30,868][INFO]: Processing file: ./sorted_time/2007-12/pagecounts-20071210-070000.gz
[2018-05-18 03:48:33,129][INFO]: Added DataFrame for file ./sorted_time/2007-12/pagecounts-20071210-070000.gz to list
[2018-05-18 03:48:33,130][INFO]: Processing file: ./sorted_time/2007-12/pagecounts-20071210-200000.gz
[2018-05-18 03:48:33,159][INFO]: Added DataFrame for file ./sorted_time/2007-12/pagecounts-20071210-200000.gz to list
[2018-05-18 03:48:33,160][INFO]: Processing file: ./sorted_time/2007-12/pagecounts-20071210-000000.gz
[2018-05-18 03:48:33,190][INFO]: Added DataFrame for file ./sorted_time/2007-12/pagecounts-20071210-000000.gz to list
[2018-05-18 03:48:33,190][INFO]: Processing file: ./sorted_time/2007-12/pagecounts-20071210-130000.gz
[2018-05-18 03:48:33,216][INFO]: Added DataFrame for file ./sorted_time/2007-12/pagecounts-20071210-130000.gz to list
[2018-05-18 03:48:33,216][INFO]: Processing file: ./sorted_time/2007-12/pagecounts-20071210-160001.gz
[2018-05-18 03:48:

In [2]:
df.head(10)

[Row(lang='aa', page='Enqlizxsh_-_English', views=1, reqbytes=1, timestamp=datetime.datetime(2007, 12, 10, 7, 0)),
 Row(lang='aa', page='Special:Allmessages', views=1, reqbytes=1, timestamp=datetime.datetime(2007, 12, 10, 7, 0)),
 Row(lang='aa', page='Special:Imagelist', views=1, reqbytes=1, timestamp=datetime.datetime(2007, 12, 10, 7, 0)),
 Row(lang='aa', page='User:Ae%C3%A6', views=1, reqbytes=1, timestamp=datetime.datetime(2007, 12, 10, 7, 0)),
 Row(lang='ab', page='%D0%93%D0%B5%D0%BE%D0%B3%D1%80%D0%B0%D1%84%D0%B8', views=2, reqbytes=2, timestamp=datetime.datetime(2007, 12, 10, 7, 0)),
 Row(lang='ab', page='%D0%98%D1%85%D0%B0%D0%B4%D0%BE%D1%83_%D0%B0%D0%B4%D0%B0%D2%9F%D1%8C%D0%B0', views=1, reqbytes=1, timestamp=datetime.datetime(2007, 12, 10, 7, 0)),
 Row(lang='ab', page='%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:Recentchangeslinked/2006', views=1, reqbytes=1, timestamp=datetime.datetime(2007, 12, 10, 7, 0)),
 Row(lang='ab', page='%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D

In [3]:
df.count()

78071097

In [4]:
df = df.drop('reqbytes')

In [5]:
df.dtypes

[('lang', 'string'),
 ('page', 'string'),
 ('views', 'int'),
 ('timestamp', 'timestamp')]

In [6]:
# grouped_df = df.groupby(['lang','page', df.timestamp.dt.day], as_index=False)

from pyspark.sql import functions
grouped_df = (df.select(['lang','page',functions.date_format('timestamp','yyyy-MM-dd').alias('day'), 'views'])
                .groupby(['lang','page','day'])
                .sum('views').alias('sum_views')
                )

In [7]:
grouped_df.head(10)

[Row(lang='af', page='1920', day='2007-12-10', sum(views)=13),
 Row(lang='af', page='51', day='2007-12-10', sum(views)=1),
 Row(lang='af', page='Esperanto', day='2007-12-10', sum(views)=11),
 Row(lang='af', page='New', day='2007-12-10', sum(views)=1),
 Row(lang='als', page='Weltwunder', day='2007-12-10', sum(views)=6),
 Row(lang='am', page='%E1%89%B3%E1%8A%95%E1%8B%9B%E1%8A%92%E1%8B%AB', day='2007-12-10', sum(views)=10),
 Row(lang='am', page='Category:%E1%8B%A8%E1%8A%A0%E1%8D%8D%E1%88%AA%E1%8A%AB_%E1%8A%A0%E1%8C%88%E1%88%AB%E1%89%B5', day='2007-12-10', sum(views)=5),
 Row(lang='an', page='2003', day='2007-12-10', sum(views)=22),
 Row(lang='an', page='A', day='2007-12-10', sum(views)=1),
 Row(lang='an', page='Madagascar', day='2007-12-10', sum(views)=8)]

In [8]:
from pyspark.sql.functions import col

grouped_df.select(['lang','page','day','sum(views)']).where((col("lang") == "en") & (col("page") == "Albert_Camus")).show()

+----+------------+----------+----------+
|lang|        page|       day|sum(views)|
+----+------------+----------+----------+
|  en|Albert_Camus|2007-12-10|      2908|
+----+------------+----------+----------+

